In [1]:
from pylsl import StreamInfo, StreamOutlet
import numpy as np
import pandas as pd
import time
from psychopy import visual, core, event
from glob import glob
from random import choice, random

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#name, type, channel_count, sampling rate, channel format, source_id
#Note that Markers, 1, and 0.0 cannot be altered
info = StreamInfo('CytonMarkers', 'Markers', 1, 0.0, 'int32', 'CytonMarkerID')
#make an outlet
outlet = StreamOutlet(info)
markernames = [1, 2, 3, 4]

In [3]:
#define a function to get frame on and off

#Assuming that you use a 60 Hz monitor:

#8.57 Hz corresponds to 60/8.57 = 7 frames on and 7 frames off.
#10 Hz --> 6 frames
#12 Hz --> 5 frames
#15 Hz --> 4 frames

#here assuming that the frequency is frequency of SHIFTS (8.57 shifts per seconds) 
#rather than CYCLES (8.57 on and offs per second) since the latter would be 
#impossible on a 60 Hz monitor, because you should then change the image between frame 3 and 4.

import math
def getFrames(freq):
    framerate = 60 # mywin.getActualFrameRate()
    frame = int(round(framerate / freq))
    frame_on = math.ceil(frame / 2)
    frame_off = math.floor(frame / 2)
    return frame_on, frame_off

In [4]:
def one_stimuli_blinking(frame_on, frame_off, pattern1, pattern2):
    while trialclock.getTime()<soa:
        pattern1.setAutoDraw(True)

        for frameN in range(frame_on):
            mywin.flip()

        pattern1.setAutoDraw(False)
        pattern2.setAutoDraw(True)

        for frameN in range(frame_off):
            mywin.flip()
        pattern2.setAutoDraw(False)

In [5]:
#Author: Apiporn Simapornchai
def three_stimuli_blinking(frame_on1, frame_off1, frame_on2, frame_off2, frame_on3, frame_off3, shapes, flipCount):
    looptime = math.gcd(frame_on1,math.gcd(frame_on2,frame_on3))
    
    print(frame_on1, frame_off1, frame_on2, frame_off2, frame_on3, frame_off3)
    print(flipCount)
    
    while trialclock.getTime()<soa:
        if(flipCount == 0 or (flipCount%frame_on1 ==0 and flipCount%(frame_on1*2) !=0)):
            shapes[0].setAutoDraw(True)
            shapes[1].setAutoDraw(False)
        if(flipCount%(frame_off1*2) ==0):
            shapes[1].setAutoDraw(True)
            shapes[0].setAutoDraw(False)

        if(flipCount == 0 or(flipCount%frame_on2 ==0 and flipCount%(frame_on2*2) !=0)):
            shapes[2].setAutoDraw(True)
            shapes[3].setAutoDraw(False)
        if(flipCount%(frame_off2*2) ==0):
            shapes[3].setAutoDraw(True)
            shapes[2].setAutoDraw(False)

        if(flipCount == 0 or(flipCount%frame_on3 ==0 and flipCount%(frame_on3*2) !=0)):
            shapes[4].setAutoDraw(True)
            shapes[5].setAutoDraw(False)
        if(flipCount%(frame_off3*2) ==0):
            shapes[5].setAutoDraw(True)
            shapes[4].setAutoDraw(False)

        for frameN in range(looptime):
            mywin.flip()
            flipCount+=1
            print(flipCount)

In [6]:
#setting params
mywin = visual.Window([1920, 1080], fullscr=False)

soa = 3  #stimulus onset asynchrony
iti = 1  #inter trial interval

test_freq = [6, 10, 15]  #, 15]
stimuli_seq = [0,1,2] * 5  #five trials for each freq in test_freq
freq_len = len(test_freq)

frame_on1, frame_off1 = getFrames(test_freq[0])
frame_on2, frame_off2 = getFrames(test_freq[1])
frame_on3, frame_off3 = getFrames(test_freq[2])

count = 0
trialclock = core.Clock()

frame_on = 0
frame_off = 0

patternup1 = visual.GratingStim(mywin, tex=None, sf=0, size=0.3,
    name='pattern1', autoLog=False, color=[1,1,1], pos=(0, 0.5))

patternup2 = visual.GratingStim(mywin, tex=None, sf=0, size=0.3,
    name='pattern2', autoLog=False, color=[-1,-1,-1], pos=(0, 0.5))

patternright1 = visual.GratingStim(mywin, tex=None, sf=0, size=0.3,
    name='pattern1', autoLog=False, color=[1,1,1], pos=(0.5, 0))

patternright2 = visual.GratingStim(mywin, tex=None, sf=0, size=0.3,
    name='pattern2', autoLog=False, color=[-1,-1,-1], pos=(0.5, 0))

#patterndown1 = visual.GratingStim(mywin, tex=None, sf=0, size=0.3,
#    name='pattern1', autoLog=False, color=[1,1,1], pos=(0, -0.5))

#patterndown2 = visual.GratingStim(mywin, tex=None, sf=0, size=0.3,
#    name='pattern2', autoLog=False, color=[-1,-1,-1], pos=(0, -0.5))

patternleft1 = visual.GratingStim(mywin, tex=None, sf=0, size=0.3,
    name='pattern1', autoLog=False, color=[1,1,1], pos=(-0.5, 0))

patternleft2 = visual.GratingStim(mywin, tex=None, sf=0, size=0.3,
    name='pattern2', autoLog=False, color=[-1,-1,-1], pos=(-0.5, 0))

shapes = [patternup1, patternup2, patternright1, patternright2, patternleft1, patternleft2]

#fixation cross
fixation = visual.ShapeStim(mywin, 
    vertices=((0, -0.5), (0, 0.5), (0,0), (-0.5,0), (0.5, 0)),
    lineWidth=5,
    closeShape=False,
    lineColor="white"
)


In [7]:
#running the actual experiment
while True:
    message = visual.TextStim(mywin, text='Start recording and press space to continue')
    message.draw()
    mywin.flip()
    keys = event.getKeys()
    if 'space' in keys:  # If space has been pushed
        message.setText = ''
        message.draw()
        mywin.flip()  
        
        fixation.draw()
        mywin.flip() #refresh
        core.wait(iti)
        mywin.flip()
        
        while count < len(stimuli_seq):
            #draw the stimuli and update the window
            print("freq: ", test_freq[count%freq_len])
            #print("frameon-off: ", frame_on, frame_off)
            print("markername: ", markernames[count%freq_len])
            print("======")
            
            outlet.push_sample([markernames[count%freq_len]])  #(x, timestamp)
            
            flipCount = 0
            #one_stimuli_blinking(frame_on, frame_off, shapes[count%freq_len*2], shapes[count%freq_len*2+1])
            three_stimuli_blinking(frame_on1, frame_off1, frame_on2, frame_off2, frame_on3, frame_off3, shapes, flipCount)
                
            #clean black screen off
            mywin.flip()
            #draw fixation
            #fixation.draw()
            #mywin.flip() #refresh
            #wait certain time for next trial
            core.wait(iti)
            #clear fixation
            mywin.flip()
            #reset clock for next trial
            trialclock.reset()    
            #count number of trials
            count+=1
        break;
            
mywin.close()  #do not delete, otherwise, the window will not turn off

freq:  6
markername:  1
5 5 3 3 2 2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
freq:  10
markername:  2
5 5 3 3 2 2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184

85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
freq:  10
markername:  2
5 5 3 3 2 2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
